In [26]:
import pandas as pd
import numpy as np
import re
# from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [16]:
import pandas as pd
train=pd.read_csv("training_twitter_x_y_train.csv")
test=pd.read_csv("test_twitter_x_test.csv")
train.isnull().sum()

tweet_id                      0
airline_sentiment             0
airline                       0
airline_sentiment_gold    10949
name                          0
negativereason_gold       10956
retweet_count                 0
text                          0
tweet_coord               10204
tweet_created                 0
tweet_location             3550
user_timezone              3577
dtype: int64

In [17]:
drop_cols = ['airline_sentiment_gold','name','tweet_id', 'retweet_count','tweet_created','user_timezone','tweet_coord','tweet_location']
train.drop(drop_cols, axis = 1, inplace=True)
test.drop(drop_cols, axis = 1, inplace=True)

In [18]:
train

,airline_sentiment,airline,negativereason_gold,text
0,negative,Southwest,NaN,"@SouthwestAir I am scheduled for the morning, ..."
1,positive,Southwest,NaN,@SouthwestAir seeing your workers time in and ...
2,positive,United,NaN,@united Flew ORD to Miami and back and had gr...
3,negative,Southwest,NaN,@SouthwestAir @dultch97 that's horse radish 😤🐴
4,negative,United,NaN,@united so our flight into ORD was delayed bec...
...,...,...,...,...
10975,neutral,American,NaN,@AmericanAir followback
10976,positive,United,NaN,@united thanks for the help. Wish the phone re...
10977,negative,US Airways,NaN,@usairways the. Worst. Ever. #dca #customerser...
10978,negative,US Airways,NaN,@nrhodes85: look! Another apology. DO NOT FLY ...


In [19]:
test

,airline,negativereason_gold,text
0,American,NaN,@AmericanAir In car gng to DFW. Pulled over 1h...
1,American,NaN,"@AmericanAir after all, the plane didn’t land ..."
2,Southwest,NaN,@SouthwestAir can't believe how many paying cu...
3,US Airways,NaN,@USAirways I can legitimately say that I would...
4,American,NaN,@AmericanAir still no response from AA. great ...
...,...,...,...
3655,US Airways,NaN,@USAirways Been stuck for 40+ minutes due to l...
3656,US Airways,NaN,@USAirways 4 hours... 4 hours... FOUR HOURS. ...
3657,Virgin America,NaN,Nice RT @VirginAmerica: The man of steel might...
3658,American,NaN,@AmericanAir Aww Thanks AA..DFW was on GMA up ...


In [25]:
from nltk.corpus import stopwords
import string
punctuation=string.punctuation
stops = stopwords.words('english')
stops += list(punctuation)
stops += ['flight','airline','flights','AA']
stops

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [27]:
abbreviations = {'ppl': 'people','cust':'customer','serv':'service','mins':'minutes','hrs':'hours','svc': 'service',
           'u':'you','pls':'please'}

train_index = train[~train.negativereason_gold.isna()].index
test_index = test[~test.negativereason_gold.isna()].index

for index, row in train.iterrows():
    tweet = row.text
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet) #remove links
    tweet = re.sub('@[^\s]+','',tweet) #remove usernames
    tweet = re.sub('[\s]+', ' ', tweet) #remove additional whitespaces
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) #replace #word with word
    tweet = tweet.strip('\'"') #trim tweet
    words = []
    for word in tweet.split():
#         if not hasNumbers(word):
        if word.lower() not in stops:
            if word in list(abbreviations.keys()):
                words.append(abbreviations[word])
            else:
                words.append(word.lower())   
    tweet = " ".join(words)
    tweet = " %s %s" % (tweet, row.airline)
    row.text = tweet
    if index in train_index:
        row.text = " %s %s" % (row.text, row.negativereason_gold)

for index, row in test.iterrows():
    tweet = row.text
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet) #remove links
    tweet = re.sub('@[^\s]+','',tweet) #remove usernames
    tweet = re.sub('[\s]+', ' ', tweet) #remove additional whitespaces
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) #replace #word with word
    tweet = tweet.strip('\'"') #trim tweet
    words = []
    for word in tweet.split(): 
#         if not hasNumbers(word):
        if word.lower() not in stops:
            if word in list(abbreviations.keys()):
                words.append(abbreviations[word])
            else:
                words.append(word.lower())
    tweet = " ".join(words)
    tweet = " %s %s" % (tweet, row.airline)
    row.text = tweet
    if index in test_index:
        row.text = " %s %s" % (row.text, row.negativereason_gold)

del train['negativereason_gold']
del test['negativereason_gold']

In [28]:
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

for index, row in train.iterrows():
    row.text = deEmojify(row.text)

for index, row in test.iterrows():
    row.text = deEmojify(row.text)

In [ ]:
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

for index, row in train.iterrows():
    words = row.text.split()
    new_words = []
    for word in words:
        if not hasNumbers(word):
            new_words.append(word)
    row.text = " ".join(new_words)
    
for index, row in test.iterrows():
    words = row.text.split()
    new_words = []
    for word in words:
        if not hasNumbers(word):
            new_words.append(word)
    row.text = " ".join(new_words)

In [47]:
train
# test.head()

,airline_sentiment,airline,text
0,negative,Southwest,"scheduled morning, 2 days fact, yes..not sure..."
1,positive,Southwest,seeing workers time time going beyond love fl...
2,positive,United,"flew ord miami back great crew, service legs...."
3,negative,Southwest,that's horse radish 😤🐴 Southwest
4,negative,United,"ord delayed air force one, last sbn 8:20, 5 m..."
...,...,...,...
10975,neutral,American,followback American
10976,positive,United,thanks help. wish phone reps could accomidati...
10977,negative,US Airways,the. worst. ever. dca customerservice US Airways
10978,negative,US Airways,look! another apology. fly US Airways


In [48]:
test

,airline,text
0,American,car gng dfw. pulled 1hr ago icy roads. on-hol...
1,American,"all, plane didn’t land identical worse) condi..."
2,Southwest,can't believe many paying customers left high...
3,US Airways,legitimately say would rather driven cross co...
4,American,still response aa. great job guys! American
...,...,...
3655,US Airways,stuck 40+ minutes due lavatory issues. bevera...
3656,US Airways,4 hours... 4 hours... four hours. like joke y...
3657,Virgin America,"nice rt man steel might faster, wifi – saying..."
3658,American,aww thanks aa..dfw gma am..so understand ..bt...


In [30]:
v = TfidfVectorizer(analyzer='word', max_features=3150, max_df = 0.8, ngram_range=(1,1))
train_features= v.fit_transform(train.text)
test_features=v.transform(test.text)

In [36]:
clf = LogisticRegression(C = 2.1, solver='liblinear', multi_class='auto')
clf.fit(train_features,train['airline_sentiment'])
pred = clf.predict(test_features)
with open('predictions_twitter.csv', 'w') as f:
    for item in pred:
        f.write("%s\n" % item)

In [37]:
clf = SVC(kernel="linear", C= 0.96 , gamma = 'scale')
# clf = SVC(C = 1000, gamma = 0.001)
clf.fit(train_features, train['airline_sentiment'])
pred = clf.predict(test_features)

In [38]:
with open('predictions_twitter2.csv', 'w') as f: #less accurate
    for item in pred:
        f.write("%s\n" % item)

In [40]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier()
clf.fit(train_features,train['airline_sentiment'])
pred = clf.predict(test_features)

In [41]:
with open('predictions_twitter3.csv', 'w') as f: #less accurate
    for item in pred:
        f.write("%s\n" % item)

In [44]:
predictions_twitter=pd.read_csv("predictions_twitter.csv")
predictions_twitter

,negative
0,negative
1,negative
2,negative
3,positive
4,negative
...,...
3654,negative
3655,negative
3656,negative
3657,positive


In [43]:
predictions_twitter2=pd.read_csv("predictions_twitter2.csv")
predictions_twitter2

,negative
0,negative
1,negative
2,negative
3,positive
4,negative
...,...
3654,negative
3655,negative
3656,negative
3657,positive


In [45]:
predictions_twitter3=pd.read_csv("predictions_twitter3.csv")
predictions_twitter3

,negative
0,negative
1,negative
2,negative
3,positive
4,negative
...,...
3654,negative
3655,negative
3656,negative
3657,positive
